## **Imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras import models
from tensorflow.keras import layers

## **Basic Preprocessing**

In [ ]:
def preprocess(x):
    x.set_index('id', inplace=True)
    x.drop(labels=['year', 'fw_start','fw_end','country','c_abrv'], axis=1, inplace=True) # Remove 'country', 'c_abrc' ?
    categorical_cols = x.select_dtypes(include=['object']).columns
    x[categorical_cols] = x[categorical_cols].astype('category')
    # x = pd.get_dummies(x, columns = categorical_cols)
    return x

# True Setup
X = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y = pd.read_csv('Y_train.csv')
X = preprocess(X)
X_test = preprocess(X_test)

y.set_index('id', inplace=True)
y.loc[y['label']==-1] = 0
y = y['label']

# Handle Missing Values
naIndices = X.index[X.isna().any(axis=1)]
X = X.drop(naIndices)
y = y.drop(naIndices)

## **Feature Engineering**

Top 500 Features from Random Forest

In [ ]:
# Train RF
randomForestModel = RandomForestClassifier(max_features=int(len(X.columns)**(1/2)), random_state=0, oob_score=True)
randomForestModel.fit(X, y)

# Extract & Sort Features by Importance
featureImportance = randomForestModel.feature_importances_

i = featureImportance.argsort()
features = X.columns[i]
importance = featureImportance[i]

# Visualize Top 10
features = features[-10:]
importance = importance[-10:]

plt.figure(figsize=(10, 6))
plt.barh(range(len(features)), importance,color='skyblue')
plt.yticks(range(len(features)), features)
plt.xlabel('Feature Influence')
plt.ylabel('Features')
plt.title('Top 10 Variable Influence for RF model')

plt.tight_layout()

# Save
featureImportancesRF = {'Feature': features, 'Feature Influence': importance}
pd.DataFrame(featureImportancesRF).sort_values('Feature Influence', ascending=False).to_excel('important_features_rf.xlsx')

Top 0.5 Importance Variables from Gradient Boosting

In [ ]:
# Train GB
defaultGBModel.fit(X, y)
importance_scores = defaultGBModel.feature_importances_

# Get feature names
feature_names = X.columns

# Create a dataframe to store feature names and their importance scores
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance_scores})

# Sort the dataframe by importance score in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# top_10_features = feature_importance_df.head(10)
top_features = []
importance_sum = 0

# Iterate through the sorted feature importance list
for index, row in feature_importance_df.iterrows():
    # Add the feature and its importance score to the top features list
    top_features.append({'Feature': row['Feature'], 'Importance': row['Importance']})
    # Accumulate the importance score
    importance_sum += row['Importance']
    # Check if the sum exceeds 0.5
    # if importance_sum > 0.5:
    #     break

# Convert the list of dictionaries to a DataFrame
top_features_df = pd.DataFrame(top_features)
top_features_df = top_features_df.iloc[:10]

# Plot feature importance for the top 10 features
plt.figure(figsize=(10, 6))
plt.barh(top_features_df['Feature'], top_features_df['Importance'], color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.title('Top 10 Feature Importance from XGBoost')
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.show()

# Save
top_features_df.to_excel('top_features.xlsx')

## **Models**

### XGBoost

In [ ]:
# Hyperparameter Tuning
param_grid = {
     'max_depth': [5,9,12],                # Maximum depth of the trees
     'learning_rate': [0.01, 0.1, 0.3],    # Learning rate
     'subsample': [0.5, 0.7],              # Subsample ratio of the training instances
     'colsample_bytree': [0.5, 0.7, 0.9],  # Subsample ratio of columns when constructing each tree
     'gamma': [0, 0.1, 1]                  # Minimum loss reduction required to make a further partition on a leaf node
 }

grid_search = GridSearchCV(estimator=boostingModel, param_grid=param_grid, cv=5, scoring='neg_log_loss')
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Default Variables
params = {
    'objective': 'multi:softmax',  # Use softmax for multi-class classification
    'num_class': 5,  # Number of classes
    'eval_metric': 'mlogloss',  # Use multi-class log loss as evaluation metric
    'eta': 0.01,  # Learning rate
    'max_depth': 8,  # Maximum depth of a tree
    'subsample': 0.8,  # Subsample ratio of the training instances
    'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree
    'gamma': 0.01,  # Minimum loss reduction required to make a further partition on a leaf node
    'enable_categorical': True,
    'alpha': 3.5,
    'lambda': 3.5,
    'min_child_weight': 1,
    'n_jobs':4,
    'n_estimators':2000,
    'device':'cuda'
}
boostingModel_allVar = XGBClassifier(**params)
scores_allVar = cross_val_score(boostingModel_allVar, X, y, cv=4, scoring="neg_log_loss")
print(np.mean(scores_allVar))

# Top 500 Variables from Random Forest
top_features = pd.read_excel('important_features_rf.xlsx').iloc[:500]
top_features = top_features['Feature'].values

params = {
    'objective': 'multi:softmax',  # Use softmax for multi-class classification
    'num_class': 5,  # Number of classes
    'eval_metric': 'mlogloss',  # Use multi-class log loss as evaluation metric
    'eta': 0.01,  # Learning rate
    'max_depth': 8,  # Maximum depth of a tree
    'subsample': 0.8,  # Subsample ratio of the training instances
    'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree
    'gamma': 0.01,  # Minimum loss reduction required to make a further partition on a leaf node
    # 'enable_categorical': True,
    'alpha': 3, # L1 regularization term
    'lambda': 3, # L2 regularization term
    'min_child_weight': 4, 
    'n_estimators':2000, # Iterations/Trees
    'device':'cuda'

}
boostingModel_RFVar = XGBClassifier(**params)
scores_RFVar = cross_val_score(boostingModel_RFVar, X, y, cv=4, scoring="neg_log_loss")
print(np.mean(scores_RFVar))

# Top 0.5 Importance Variables from Gradient Boosting
top_features = pd.read_excel('top_features.xlsx')
top_features = top_features['Feature'].values

params = {
    'objective': 'multi:softmax',  # Use softmax for multi-class classification
    'num_class': 5,  # Number of classes
    'eval_metric': 'mlogloss',  # Use multi-class log loss as evaluation metric
    'eta': 0.01,  # Learning rate
    'max_depth': 8,  # Maximum depth of a tree
    'subsample': 0.8,  # Subsample ratio of the training instances
    'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree
    'gamma': 0.01,  # Minimum loss reduction required to make a further partition on a leaf node
    'enable_categorical': True,
    'alpha': 3.5,
    'lambda': 3.5,
    'min_child_weight': 1,
    'n_jobs':4,
    'n_estimators':2000,
    'device':'cuda'

}
boostingModel_GBVar = XGBClassifier(**params)
scores_GBVar = cross_val_score(boostingModel_GBVar, X, y, cv=4, scoring="neg_log_loss")
print(np.mean(scores_GBVar))

### SVM

In [ ]:
# Define preprocessing steps including imputation
preprocessor = ColumnTransformer(
    transformers=[
        ('num', make_pipeline(SimpleImputer(strategy='mean'), StandardScaler()), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Define the pipeline including preprocessing and SVM
svm_pipeline = make_pipeline(preprocessor, SVC())

# Train the SVM model
SVMModel = svm_pipeline.fit(X, y)

scores_SVM = cross_val_score(SVMModel, X, y, cv=4, scoring="neg_log_loss")
print(np.mean(scores_SVM))

### Neural Network

In [ ]:
model = models.Sequential()
model.add(layers.Dense(1000, activation='relu', input_shape=(5,)))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(1000, activation='relu'))
# model.add(layers.Dropout(0.2))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.025, momentum=0.8),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X,
                    y,
                    epochs=250,
                    batch_size=1000,
                    validation_data=(X_test, y))

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

### Stacking: XGBoost + Random Forest

In [ ]:
# Train Random Forest and XGBoost models
randomForestModel = RandomForestClassifier(max_features=int(np.sqrt(X_train.shape[1])), random_state=0, oob_score=True)
randomForestModel.fit(X_train, y_train)

params = {
    'objective': 'multi:softmax',  # Use softmax for multi-class classification
    'num_class': 5,  # Number of classes
    'eval_metric': 'mlogloss',  # Use multi-class log loss as evaluation metric
    'eta': 0.01,  # Learning rate
    'max_depth': 8,  # Maximum depth of a tree
    'subsample': 0.8,  # Subsample ratio of the training instances
    'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree
    'gamma': 0.01,  # Minimum loss reduction required to make a further partition on a leaf node
    'alpha': 3,
    'lambda': 3,
    'min_child_weight': 4,
    'n_jobs': 4,
    'n_estimators': 2000
}

boostingModel = XGBClassifier(**params)
boostingModel.fit(X_train, y_train)

# Generate predictions on the validation data
randomForestPred = randomForestModel.predict_proba(X_test)
xgboostPred = boostingModel.predict_proba(X_test)
rf_pred_proba_train = randomForestModel.predict_proba(X_train)
xgb_pred_proba_train = boostingModel.predict_proba(X_train)

# Combine predictions into a new feature matrix
stacked_X_test = np.concatenate((randomForestPred, xgboostPred), axis=1)

# Concatenate the predicted probabilities from both models along axis 1 (columns)
stacked_X_train = np.concatenate((rf_pred_proba_train, xgb_pred_proba_train), axis=1)

# Train a meta-model (Logistic Regression) on the combined predictions
meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(stacked_X_train, y_train)  # Assuming stacked_X_train is defined

# Use the meta-model to make predictions on the test data
stacked_pred_proba = meta_model.predict_proba(stacked_X_test)

# Define the cross-validation strategy (Stratified K-Folds with k=5)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and calculate log loss scores
log_loss_scores = cross_val_score(meta_model, stacked_X_train, y_train, scoring='neg_log_loss', cv=cv)

# Print log loss scores
print("Log Loss Scores:", log_loss_scores)

## **Comparisons**

In [ ]:
# rfScores = cross_val_score(randomForestModel, X, y, cv=4, scoring='neg_log_loss')
import matplotlib.pyplot as plt
data = [rfScores, scores_default, scores_allVar, scores_RFVar, scores_GBVar, svm_xgboosting, xgboosting_rf]

plt.boxplot(data)

# Add labels
plt.ylabel('Negative Log Loss')
plt.title('Log loss of 5 XGBoost models and RF model')

# Customize x-axis tick labels
plt.xticks(range(1, len(data) + 1), ['RF all var','Default all var', 'Tuned all var', 'Top 500 RF Var', 'Top GB Var', 'SVM + XGB'])

# Show the plot
plt.tight_layout()

# rfScores = cross_val_score(randomForestModel, X, y, cv=4, scoring='neg_log_loss')
import matplotlib.pyplot as plt

# Provided data
svm_xgboosting = [-0.87616632, -0.86292975, -0.86713181, -0.87255519, -0.83238594]
xgboosting_rf = [-0.86726267, -0.85745867, -0.86215765, -0.85209438, -0.83243321]
svm = [0.88714105, 0.88489864, 0.87195326, 0.8755607,  0.89217313]

data = [rfScores, scores_default, scores_allVar, scores_RFVar, scores_GBVar, svm_xgboosting, xgboosting_rf]

plt.boxplot(data)

# Add labels
plt.ylabel('Negative Log Loss')
plt.title('Log loss of 5 XGBoost models and RF model')

# Customize x-axis tick labels
plt.xticks(range(1, len(data) + 1), ['RF all var','Default all var', 'Tuned all var', 'Top 500 RF Var', 'Top GB Var', 'SVM + XGB'])

# Show the plot
plt.tight_layout()